<div>
<img src="./images/sunpy_logo.png" width="500" align="left"/>
</div>

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from sunpy.time import parse_time
import numpy as np
import matplotlib.pyplot as plt
import glob
import sunpy.map
from sunpy.coordinates import frames, get_body_heliographic_stonyhurst, get_horizons_coord, transform_with_sun_center
from astropy.visualization import AsymmetricPercentileInterval, ImageNormalize, LogStretch



# 3. Coordinates

SunPy uses [`astropy.coordinates`](https://docs.astropy.org/en/stable/coordinates/index.html) to represent points in physical space. This applies to both points in 3D space and projected coordinates in images.

The `sunpy.coordinates` sub-package contains:

* A robust framework for working with solar-physics coordinate systems
* Functions to obtain the locations of solar-system bodies (`sunpy.coordinates.ephemeris`)
* Functions to calculate Sun-specific coordinate information (`sunpy.coordinates.sun`)


In this notebook we'll introduce some of the powerful functionality available within sunpy

## Coordinate frameword
`sunpy` extends the `astropy.coordinates` framework by adding additional solar-specific coordinate frames and the accompanying transformations between them.

<div>
<img src="./images/coordinates.svg"/>
</div>

### Creating coordinates 

We deal with coordinates by using astropy's [`SkyCoord`](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord) class. 

We have already seem some introduction to this in the previous notebook but lets extend this here!


A **coordinate** combines position data with a coordinate frame, and a SkyCoord object is created by passing in positions with specified units and a coordinate frame. Above in the imports cell we've imported [`sunpy.coordinates.frames`](https://docs.sunpy.org/en/stable/code_ref/coordinates/index.html#supported-coordinate-systems) which allow us to use solar physics specific frames such as Helioprojective, Heliographic Stonyhurst, Heliocentric etc. 

Lets create a point on the Sun in lat and long in the Heliographic Stonyhurst coordinate system

In [ ]:
# longitude, latitude
hgs_coord = SkyCoord(10*u.deg, 20*u.deg, obstime="2017-08-01", frame=frames.HeliographicStonyhurst)  
hgs_coord

In [ ]:
print(f"""Longitude: {hgs_coord.lon}
Latitude: {hgs_coord.lat}
Distance from Sun center: {hgs_coord.radius}""")

In [ ]:
hgs_coord_xyz = SkyCoord(hgs_coord, representation_type='cartesian')
hgs_coord_xyz

We can then transform this coordinate to the any defined coordinate frame defined in astropy or sunpy. Lets transform it to the Helioprojective frame (which is observer-based)

In [ ]:
hgs_coord.transform_to(frames.Helioprojective(observer="earth"))

We can also convert this to other coordinate systems outside the solar-specific ones - for examples ICRS

In [ ]:
hgs_coord.transform_to("icrs")

## An important note about observer based frames

Some coordinate frames are defined based on the position of the observer e.g. the Helioprojective and Heliocentric frames. Hence it's important to think about this - particularly when transforming points between coordinate systems. 
This is shown above when transforming to Helioprojective we needed to pass an `observer` keyword. Similarly, its important that the `obstime` is given also!

For example, lets define a point on the Sun in Helioprojective and see what that equivalent point would be from another observer - say Mars!

In [ ]:
obstime = "2022-05-02 00:00"

In [ ]:
hpc_coord = SkyCoord(0*u.arcsec, 0*u.arcsec, observer="earth", 
                     obstime=obstime, frame=frames.Helioprojective)
hpc_coord

In [ ]:
print(hpc_coord.Tx, hpc_coord.Ty)

In [ ]:
mars_hpc_coord = hpc_coord.transform_to(frames.Helioprojective(observer="mars"))
mars_hpc_coord

In [ ]:
print(mars_hpc_coord.Tx, mars_hpc_coord.Ty)

## Positions of solar system bodies
`sunpy.coordinates` provides functions to obtain the coordinates of solar-system bodies.
The function `get_body_heliographic_stonyhurst` which will return the location of the solar-system body in the `HeliographicStonyhurst` frame.

For other solar-system bodies (e.g., major man-made spacecraft or comets), you can use `get_horizons_coord()`, which queries JPL HORIZONS:

In [ ]:
earth_pos = get_body_heliographic_stonyhurst("earth", "2022-03-28")
mars_pos = get_body_heliographic_stonyhurst("mars", "2022-03-28")

In [ ]:
print(mars_pos)

For other solar-system bodies (e.g., major man-made spacecraft or comets), you can use get_horizons_coord(), which queries JPL HORIZONS:

In [ ]:
solo_pos = get_horizons_coord("solar orbiter", "2022-03-28")

## Plotting positions of spacecraft

Lets plot the positions of different spacecraft over the recent Solar Orbiter perihelion!

In [ ]:
perihelion_time = parse_time("2022-03-26")
perihelion_seq = perihelion_time + np.arange(-30, 30)*u.day


In [ ]:
solo_coord = get_horizons_coord("solar orbiter", perihelion_seq)
psp_coord = get_horizons_coord("psp", perihelion_seq)
sdo_coord = get_horizons_coord("sdo", perihelion_seq)

In [ ]:
fig = plt.figure(dpi=120)
ax = fig.add_subplot(projection='polar')

# Transform to HGS
psp_coord_hgs = psp_coord.heliographic_stonyhurst
solo_coord_hgs = solo_coord.heliographic_stonyhurst
sdo_coord_hgs = sdo_coord.heliographic_stonyhurst


ax.plot(psp_coord_hgs.lon.to('rad'), psp_coord_hgs.radius,
        '.', markersize=5, label='PSP')
ax.plot(solo_coord_hgs.lon.to('rad'), solo_coord_hgs.radius,
        '.', markersize=5, label='SolO')
ax.plot(sdo_coord_hgs.lon.to('rad'), sdo_coord_hgs.radius,
        '.', markersize=5, label='SDO')


ax.legend(loc='lower right')
ax.set_theta_zero_location("S")
ax.set_title('Positions in Heliographic Stonyhurst (HGS)')

# Reproject Maps from one observer to another


## Reproject AIA to Solar Orbiter field of view

In [ ]:
aia_files = glob.glob("./AIA/*.fits"); aia_files.sort()
eui_files = glob.glob("./EUI/*.fits"); eui_files.sort()

In [ ]:
aia_map = sunpy.map.Map(aia_files[2])
eui_map = sunpy.map.Map(eui_files[2])

In [ ]:
eui_map.plot_settings["norm"] = ImageNormalize(vmin=0, vmax=300, stretch=LogStretch())


We can plot the solar limb as seen from EUI on the AIA map

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax1 = fig.add_subplot(projection=aia_map)
aia_map.plot(axes=ax1, vmin=0, vmax=200)
aia_map.draw_limb(axes=ax1, color='white')
eui_map.draw_limb(axes=ax1, color='blue', label="EUI limb")



In [ ]:
fig = plt.figure(figsize=(8, 8))

ax1 = fig.add_subplot(projection=eui_map)
eui_map.plot(axes=ax1)
eui_map.draw_limb(axes=ax1, color='white')
aia_map.draw_limb(axes=ax1, color='blue', label="AIA limb")
plt.legend()



## Reproject AIA to field of view of Solo

In [ ]:
aia_map = aia_map.resample((512, 512)*u.pix)

In [ ]:
from reproject import reproject_interp
from astropy.wcs import WCS

In [ ]:
outshape = (1500, 1500)# aia_map.data.shape
ref_coord = SkyCoord(0*u.arcsec, 0*u.arcsec,
                     frame='helioprojective', obstime=eui_map.date, observer=eui_map.observer_coordinate)

# Create a FITS WCS header for the reference coordinate and frame
header = sunpy.map.make_fitswcs_header(
    outshape,
    ref_coord,
    scale=u.Quantity(aia_map.scale),
)
header['rsun_ref'] = aia_map.meta['rsun_ref']


In [ ]:
outmap = aia_map.reproject_to(header)
outmap.plot_settings = aia_map.plot_settings

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection=outmap)
outmap.plot(vmin=0, vmax=100)
outmap.draw_limb(color='k')
ax.set_title("AIA from view of Solar Orbiter")